In [2]:
%run sim_env_setup.ipynb
%run kinematics.ipynb

1.942890293094024e-16 -3.3306690738754696e-16
5.551115123125783e-16 -8.881784197001252e-16
-4.440892098500626e-16 9.992007221626409e-16
-5.551115123125783e-16 6.661338147750939e-16
-3.3306690738754696e-16 5.551115123125783e-16
-6.661338147750939e-16 1.1102230246251565e-15
4.440892098500626e-16 -7.771561172376096e-16
-3.3306690738754696e-16 5.551115123125783e-16
-4.884981308350689e-15 7.327471962526033e-15
4.440892098500626e-16 -6.661338147750939e-16
-1.1102230246251565e-16 2.220446049250313e-16
5.551115123125783e-16 -7.771561172376096e-16
-6.591949208711867e-17 2.220446049250313e-16
3.164135620181696e-15 -4.829470157119431e-15
-6.661338147750939e-16 9.992007221626409e-16
8.881784197001252e-16 -1.3322676295501878e-15
-9.992007221626409e-16 1.7763568394002505e-15
-3.3861802251067274e-15 5.162537064506978e-15
-3.4416913763379853e-15 5.218048215738236e-15
-3.885780586188048e-16 5.551115123125783e-16
-1.1102230246251565e-16 0.0
4.996003610813204e-16 -6.661338147750939e-16
-2.359223927328457

1.4432899320127035e-15 -2.4424906541753444e-15
-8.770761894538737e-15 1.3100631690576847e-14
-4.440892098500626e-16 6.661338147750939e-16
-2.220446049250313e-16 4.440892098500626e-16
-5.551115123125783e-17 0.0
-7.771561172376096e-16 1.2212453270876722e-15
0.0 0.0
1.1102230246251565e-15 -1.887379141862766e-15
1.2212453270876722e-15 -1.9984014443252818e-15
-1.942890293094024e-15 3.0531133177191805e-15
1.4654943925052066e-14 -2.2093438190040615e-14
-4.218847493575595e-15 6.328271240363392e-15
5.551115123125783e-17 -2.220446049250313e-16
6.661338147750939e-16 -9.992007221626409e-16
-3.0253577421035516e-15 4.6629367034256575e-15
6.106226635438361e-16 -9.992007221626409e-16
-6.328271240363392e-15 9.547918011776346e-15
1.1102230246251565e-16 -1.1102230246251565e-16
-9.853229343548264e-16 1.5543122344752192e-15
0.0 0.0
0.0 0.0
-9.992007221626409e-16 1.5543122344752192e-15
8.881784197001252e-16 -1.2212453270876722e-15
7.216449660063518e-16 -1.1657341758564144e-15
-8.881784197001252e-16 1.332267

In [3]:
import numpy as np
# Run this command to disable the motors
p.setJointMotorControlArray(scara, [1, 2], p.VELOCITY_CONTROL, forces=[0, 0])

In [4]:
p.setJointMotorControl2(scara, 1, p.TORQUE_CONTROL, force=-50)

for _ in range(1000):
    p.stepSimulation()
    
p.getJointState(scara, 1)

(-0.04443325490477883,
 -0.04351272342656368,
 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 0.0)

In [5]:
# Read position at every time step and use proporitonal control to reach the position
dt = 0.001
p_gain = 2
d_gain = 5
error = 0
error_old = 0
desired_pos = 1.5
for _ in range(1000):
    pos, _, _, _ = p.getJointState(scara,1)
    error = desired_pos - pos
    error_d = (error - error_old)/dt
    control_force = p_gain * error + d_gain * error_d
    p.setJointMotorControl2(scara, 1, p.TORQUE_CONTROL, force=control_force)
    p.stepSimulation()    

In [6]:
# Read position at every time step and use proporitonal control to reach the position
dt = 0.001
p_gain = 2
d_gain = 5
error = 0
error_old = 0
desired_pos = np.array([1.5, 1.5])
for _ in range(100000):
    pos1, _, _, _ = p.getJointState(scara,1)
    pos2, _, _, _ = p.getJointState(scara,2)
    pos = np.array([pos1, pos2])
    error = desired_pos - pos
    error_d = (error - error_old)/dt
    control_force = p_gain * error + d_gain * error_d
    p.setJointMotorControlArray(scara, [1,2], p.TORQUE_CONTROL, forces=control_force)
    p.stepSimulation()

In [7]:
def control(theta1, theta2):
    # Read position at every time step and use proporitonal control to reach the position
    dt = 0.001
    p_gain = 2
    d_gain = 5
    error = 0
    error_old = 0
    desired_pos = np.array([theta1, theta2])
    for _ in range(100000):
        pos1, _, _, _ = p.getJointState(scara,1)
        pos2, _, _, _ = p.getJointState(scara,2)
        pos = np.array([pos1, pos2])
        error = desired_pos - pos
        error_d = (error - error_old)/dt
        control_force = p_gain * error + d_gain * error_d
        return control_force
        
        

In [9]:
# Kinematics for serial-2R

p1 = np.array([1.0, 0.5])
p2 = np.array([0.5, 1.0])
pt = p1

valid, [theta1, theta2] = inverse_kinematics(pt[0], pt[1], 1, 1)

for _ in range(10000):
    control_force = control(theta1, theta2)
    p.setJointMotorControlArray(scara, [1,2], p.TORQUE_CONTROL, forces=control_force)
    p.stepSimulation()


In [10]:
p.disconnect()